# Metaphor algorithm

1. Parse file and list objects & mapto implied by subsets, common structures/covers
2. Match structures  
    a. Match types (how to deal with mappable ones? eg. segment->linear)  
    b. Match affected and covers
3. Match representations (keeping the structure consistent)



In [95]:
import yaml
import re
from pprint import pprint

def get_spec(file_path):
  with open(file_path, 'r') as file_handle:
    spec = yaml.safe_load(file_handle)
  return spec

In [96]:
ex_spec = get_spec('video-editor.yaml')

In [97]:
def process_listable(target, func):
  if target is None:
    return

  if type(target) == list:
    for target_item in target:
      func(target_item)
  else:
    func(target)

## Parsing for objects

In [98]:
def strip_type(obj):
  return re.sub(r'\((\w|-)*\) ', '', obj)

def register_object(objects, obj):
  # -> means mapto
  # . means subset
  # / means component  # TODO: actually do this!!
  assert(type(obj) is str)
  obj = strip_type(obj)

  arrow_array = obj.split('->')
  for arrow_idx, arrow_term in enumerate(arrow_array):
    
    dot_array = arrow_term.split('.')
    for dot_idx, dot_term in enumerate(dot_array):
      subject = '.'.join(dot_array[:dot_idx + 1]) # join up to idx
      if not objects.get(subject):
        objects[subject] = set()
      
      if dot_idx > 0:
        # every sequence maps to the previous element eg. a.b.c => {a.b.c: {a.b}, a.b: {a}}
        previous = '.'.join(dot_array[:dot_idx])
        objects[subject].add(previous)
    
    if arrow_idx > 0:
      # a pair of arrows lhs->rhs => {lhs: {rhs}}
      lhs = objects[arrow_array[arrow_idx - 1]]
      rhs = arrow_array[arrow_idx]
      lhs.add(rhs)

In [99]:
# Tests
print('testing: a')
test = {}
print('expected:')
pprint({'a': set()})
print('got:')
register_object(test, 'a')
pprint(test)

print()

print('testing: a.b.c')
test = {}
print('expected:')
pprint({'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}})
print('got:')
register_object(test, 'a.b.c')
pprint(test)

print()

print('testing: a->b->c')
test = {}
print('expected:')
pprint({'a': {'b'}, 'b': {'c'}, 'c': set()})
print('got:')
register_object(test, 'a->b->c')
pprint(test)

print()

print('testing: a.b->x->z.w')

test = {}
register_object(test, 'a.b->x->z.w')
print('expected:')
print({'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}})
print('got:')
pprint(test)


testing: a
expected:
{'a': set()}
got:
{'a': set()}

testing: a.b.c
expected:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}
got:
{'a': set(), 'a.b': {'a'}, 'a.b.c': {'a.b'}}

testing: a->b->c
expected:
{'a': {'b'}, 'b': {'c'}, 'c': set()}
got:
{'a': {'b'}, 'b': {'c'}, 'c': set()}

testing: a.b->x->z.w
expected:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}
got:
{'a': set(), 'a.b': {'x', 'a'}, 'x': {'z.w'}, 'z': set(), 'z.w': {'z'}}


In [100]:
def register_struct_objects(struct, objects, process_func):
  if struct.get('type') == 'group':
    # Groups also behave as objects, so register them
    name = struct.get('name')
    if name is None:
      print('Warning! No name provided for group. Using `NO NAME PROVIDED` instead. TODO: generate ID.')
      name = 'NO NAME PROVIDED'

    if objects.get(name) is None:
      objects[name] = set()
    
    # TODO: do groups map to their elements? Really, does the transitive property apply? My hunch is no, but need to think more

  process_listable(struct.get('affects'), process_func)
  process_listable(struct.get('covered-by'), process_func)
  # TODO: relate all of the objects affected/covered by a structure. 

  for derivative in struct.get('structures', []):
    register_struct_objects(derivative, objects=objects, process_func=process_func)


def register_repr_objects(repr, objects, process_func):
  repr_type = repr.get('type', '')
  # print(repr_type)
  for repr_item in repr.get('objects', []):
    assert(len(repr_item.values()) == 1)
    repr_obj, target_objs = list(repr_item.items())[0]

    process_listable(target_objs, process_func)

    # Map every item to the associated representational object
    # eg. {message: {textbox}, author: {textbox}}
    def register_repr_maps(target):
      # note: this is a bit backwards compared to the syntax!
      if objects.get(target) is None:
        objects[target] = set()
      objects[target].add(repr_obj)
      # objects[target].add(repr_type + '/' + repr_obj)  # when we prefix the core stuff
    process_listable(target_objs, register_repr_maps)



def register_spec_objects(spec, objects={}): # {object: [mapto-targets]}
  # TODO: deal with `objects` block
  def register_object_here(target):
    register_object(objects, target)
  
  process_listable(spec.get('objects'), register_object_here)

  for struct in spec.get('structures', []):
    register_struct_objects(struct, objects, register_object_here)
    
  for repr in spec.get('representations', []):
    register_repr_objects(repr, objects, register_object_here)
  
  return objects

In [101]:
pprint(register_spec_objects(ex_spec))

{'playhead': {'vline', 'videos.in-editor/images'},
 'playhead->videos.in-editor/images': {'rect'},
 'timestamps': {'vline'},
 'tracks': {'region'},
 'videos': set(),
 'videos.in-editor': {'rect', 'videos'},
 'videos.in-editor/images': {'videos'},
 'videos/first-frame': {'regions'}}


In [102]:
core_spec = get_spec('core.yaml')
core_objs = {}

# TODO: probably need to consider prefixing gui on these
for repr_type in core_spec.get('representation-types', []):
  # print(repr_type['name'])
  register_spec_objects(repr_type, core_objs)

# register_spec_objects(core_spec, core_objs)

pprint(core_objs)

{'hlines': {'lines'},
 'lines': {'regions'},
 'points': set(),
 'rects': {'regions'},
 'regions': set(),
 'vlines': {'lines'}}


In [ ]:
def get_node_join(left, right, relations, visited=set()):
  # TODO: I bet there's 
  assert(type(left) is str)
  assert(type(right) is str)
  # print(f"{left:<8} {right:<}")

  if left in visited or right in visited:
    # To avoid cycles, abort if we've already visited a node
    # NOTE: Not like 100% sure this is clean
    print("WARNING! Cycle detected in the relations.")
    return None

  if left == right:
    return left
  
  for next_left in relations[left]:
    left_res = get_node_join(next_left, right, relations, visited=visited)
    if left_res is None:
      for next_right in relations[right]:
        return get_node_join(left, next_right, relations, visited=visited)
    else:
      return left_res

  if len(relations[right]) == 0 and len(relations[left]) == 0:
    return None


In [137]:
print(get_node_join('vlines', 'hlines', core_objs))

vlines   hlines
lines    hlines
regions  hlines
lines    lines
lines


## Prepare structure relations
We want to be able to say that two structures have the same time if they both map to a common structure (if they have common structure eg. gui.stack) and structure type.